In [1]:
# Import the libraries we will need
import pandas as pd
import numpy as np

import findspark
findspark.init()

from pyspark.sql import *
import pyspark.sql.functions as F
from pyspark.sql.types import *

# create the Spark Session
spark = SparkSession.builder.appName("Q2").getOrCreate()

In [39]:
small = spark.sparkContext.textFile("hw3-bundle/hw3-bundle/pagerank_hits/data/graph-small.txt")
# small = spark.read.csv("hw3-bundle/hw3-bundle/pagerank_hits/data/graph-small.txt", sep="\t", inferSchema=True).toDF("source", "destination")

beta = 0.8
n = 100
r = np.ones(n) / n

In [40]:
small = small.map(lambda x: (int(x.split()[0]), (int(x.split()[1])))) # (source, destination)

In [41]:
small.take(10)

[(100, 1),
 (13, 1),
 (28, 1),
 (89, 1),
 (82, 1),
 (30, 1),
 (79, 1),
 (65, 1),
 (88, 1),
 (25, 1)]

In [42]:
out_deg = small.map(lambda x: (x[0], 1)).reduceByKey(lambda v1, v2: v1 + v2).map(lambda x: (x[0], 1 / x[1]))
out_deg.take(10)

[(100, 0.08333333333333333),
 (28, 0.09090909090909091),
 (82, 0.07692307692307693),
 (30, 0.1111111111111111),
 (88, 0.125),
 (46, 0.07142857142857142),
 (50, 0.07142857142857142),
 (24, 0.1),
 (38, 0.07692307692307693),
 (44, 0.1111111111111111)]

In [43]:
M = small.join(out_deg).map(lambda x: (x[1][0], x[0], x[1][1]))
M.take(15)

[(1, 100, 0.08333333333333333),
 (14, 100, 0.08333333333333333),
 (18, 100, 0.08333333333333333),
 (27, 100, 0.08333333333333333),
 (27, 100, 0.08333333333333333),
 (31, 100, 0.08333333333333333),
 (40, 100, 0.08333333333333333),
 (42, 100, 0.08333333333333333),
 (57, 100, 0.08333333333333333),
 (63, 100, 0.08333333333333333),
 (71, 100, 0.08333333333333333),
 (76, 100, 0.08333333333333333),
 (1, 28, 0.09090909090909091),
 (1, 28, 0.09090909090909091),
 (1, 28, 0.09090909090909091)]

In [45]:
M_dot_r = M.map(lambda x: (x[0], x[2] * r[int(x[1] - 1)])).reduceByKey(lambda v1, v2: v1 + v2).sortByKey()

In [51]:
M_dot_r_np = np.array(M_dot_r.map(lambda x: x[1]).collect())

In [52]:
M_dot_r_np

array([0.03781851, 0.00303122, 0.00692929, 0.00611483, 0.01552542,
       0.00507576, 0.0034561 , 0.00628336, 0.01568176, 0.00485354,
       0.00319935, 0.00549517, 0.01916617, 0.04273532, 0.00339394,
       0.00714018, 0.00682393, 0.01729234, 0.00364358, 0.00439261,
       0.00538528, 0.01520581, 0.00303114, 0.00641667, 0.00651227,
       0.01620147, 0.03809744, 0.00385844, 0.00724733, 0.00742832,
       0.01576717, 0.00436601, 0.00486111, 0.00619755, 0.01549204,
       0.00416306, 0.00349242, 0.00743347, 0.01676198, 0.04153709,
       0.00397558, 0.0072702 , 0.00701068, 0.01553449, 0.00347712,
       0.00390727, 0.00696104, 0.02053733, 0.00368687, 0.00413131,
       0.00626984, 0.01731996, 0.04498059, 0.00603968, 0.01090909,
       0.00716306, 0.0170464 , 0.00382479, 0.00369298, 0.00623975,
       0.01523153, 0.0042702 , 0.0032619 , 0.00897902, 0.01539086,
       0.03948125, 0.00371955, 0.00633766, 0.00658766, 0.01532331,
       0.00394444, 0.0033017 , 0.00686852, 0.01613279, 0.00363

In [53]:
r = beta * M_dot_r_np + (1 - beta) / n

In [56]:
r

array([0.03514138, 0.00550444, 0.00749495, 0.0070644 , 0.0143741 ,
       0.00605002, 0.00404742, 0.00813281, 0.01352938, 0.00780504,
       0.00493804, 0.00507789, 0.01696533, 0.03586677, 0.00636046,
       0.00568594, 0.00587875, 0.01331689, 0.00546612, 0.00485897,
       0.0062085 , 0.01418511, 0.00447314, 0.00623798, 0.00678083,
       0.01402686, 0.0331302 , 0.00661586, 0.00674118, 0.00665952,
       0.01210266, 0.00481402, 0.00502957, 0.00608728, 0.01183148,
       0.00531844, 0.00371428, 0.00684048, 0.01239903, 0.03383064,
       0.00727963, 0.01109913, 0.00660371, 0.01201036, 0.00437439,
       0.00511566, 0.00606424, 0.02574298, 0.00535657, 0.00423294,
       0.00765944, 0.01325418, 0.03786861, 0.0117551 , 0.00915083,
       0.00783616, 0.01733145, 0.00494735, 0.00344426, 0.00880096,
       0.01788219, 0.00722944, 0.00404528, 0.00729548, 0.01760235,
       0.02942981, 0.00985418, 0.00540312, 0.00637803, 0.01132412,
       0.00673147, 0.00417022, 0.00740847, 0.01534616, 0.00614

In [59]:
small = spark.sparkContext.textFile("hw3-bundle/hw3-bundle/pagerank_hits/data/graph-small.txt")
small = small.map(lambda x: (int(x.split()[0]), (int(x.split()[1])))) # (source, destination)
out_deg = small.map(lambda x: (x[0], 1)).reduceByKey(lambda v1, v2: v1 + v2).map(lambda x: (x[0], 1 / x[1]))
M = small.join(out_deg).map(lambda x: (x[1][0], x[0], x[1][1]))

beta = 0.8
n = 100
r = np.ones(n) / n

for i in range(40):
    M_dot_r = M.map(lambda x: (x[0], x[2] * r[int(x[1] - 1)])).reduceByKey(lambda v1, v2: v1 + v2).sortByKey()
    M_dot_r_np = np.array(M_dot_r.map(lambda x: x[1]).collect())
    r = beta * M_dot_r_np + (1 - beta) / n

    print(f"At iteration {i}, top 5 scores are {np.sort(r)[-5:]}")

At iteration 0, top 5 scores are [0.03247795 0.033585   0.03522967 0.03618825 0.03798447]
At iteration 1, top 5 scores are [0.0318378  0.03188942 0.03578222 0.03738246 0.0384674 ]
At iteration 2, top 5 scores are [0.03334838 0.03395095 0.03527318 0.03546707 0.03780861]
At iteration 3, top 5 scores are [0.03310996 0.03376326 0.03498368 0.03600937 0.03799823]
At iteration 4, top 5 scores are [0.03313224 0.03383663 0.03518425 0.03586144 0.03785768]
At iteration 5, top 5 scores are [0.03312542 0.03383009 0.03513543 0.03586724 0.03786972]
At iteration 6, top 5 scores are [0.03313164 0.03383172 0.0351419  0.03586616 0.03786869]
At iteration 7, top 5 scores are [0.0331298  0.03383042 0.03514125 0.03586707 0.03786845]
At iteration 8, top 5 scores are [0.03313036 0.03383078 0.03514148 0.0358666  0.03786857]
At iteration 9, top 5 scores are [0.03313015 0.0338306  0.03514132 0.03586683 0.03786862]
At iteration 10, top 5 scores are [0.03313021 0.03383066 0.03514141 0.03586676 0.03786862]
At iterat

In [61]:
np.argmax(r)

52

In [62]:
full = spark.sparkContext.textFile("hw3-bundle/hw3-bundle/pagerank_hits/data/graph-full.txt")
full = full.map(lambda x: (int(x.split()[0]), (int(x.split()[1])))) # (source, destination)
out_deg = full.map(lambda x: (x[0], 1)).reduceByKey(lambda v1, v2: v1 + v2).map(lambda x: (x[0], 1 / x[1]))
M = full.join(out_deg).map(lambda x: (x[1][0], x[0], x[1][1]))

beta = 0.8
n = 1000
r = np.ones(n) / n

for i in range(40):
    M_dot_r = M.map(lambda x: (x[0], x[2] * r[int(x[1] - 1)])).reduceByKey(lambda v1, v2: v1 + v2).sortByKey()
    M_dot_r_np = np.array(M_dot_r.map(lambda x: x[1]).collect())
    r = beta * M_dot_r_np + (1 - beta) / n

    print(f"At iteration {i}, top 5 scores are {np.sort(r)[-5:]}")

At iteration 0, top 5 scores are [0.00178063 0.00180125 0.00184313 0.00186635 0.0018759 ]
At iteration 1, top 5 scores are [0.00181917 0.00187874 0.00187924 0.00193098 0.00208151]
At iteration 2, top 5 scores are [0.0018393  0.00185236 0.0018951  0.0019298  0.00202104]
At iteration 3, top 5 scores are [0.00183115 0.00184855 0.00189586 0.00194235 0.0020089 ]
At iteration 4, top 5 scores are [0.00183019 0.00184973 0.00189473 0.00194597 0.00201834]
At iteration 5, top 5 scores are [0.00182986 0.0018497  0.00189399 0.00194383 0.00201781]
At iteration 6, top 5 scores are [0.00183004 0.00184942 0.00189429 0.00194384 0.0020175 ]
At iteration 7, top 5 scores are [0.00183008 0.00184952 0.00189438 0.00194382 0.00201748]
At iteration 8, top 5 scores are [0.00183003 0.00184953 0.00189436 0.00194385 0.00201742]
At iteration 9, top 5 scores are [0.00183004 0.00184955 0.00189436 0.00194384 0.00201742]
At iteration 10, top 5 scores are [0.00183004 0.00184955 0.00189436 0.00194384 0.00201742]
At iterat

In [69]:
# top 5
np.argpartition(r, -5)[-5:]

array([186, 964, 242, 262, 536], dtype=int64)

In [66]:
np.partition(r, -5)[-5:]

array([0.00183004, 0.00189436, 0.00184955, 0.00201742, 0.00194384])

In [70]:
# bottom 5
np.argpartition(r, 5)[:5]

array([423,  92, 557,  61, 407], dtype=int64)